In [104]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import xgboost
import csv as csv
from xgboost import plot_importance
from matplotlib import pyplot
from sklearn.model_selection import cross_val_score,KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV   #Perforing grid search
from scipy.stats import skew
from collections import OrderedDict
from tqdm import tqdm_notebook
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

In [9]:
df = pd.read_excel('data.xls')

In [13]:
df.head()

,Case,SMILES,MTP,diameter,petitjean,petitjeanSC,radius,VDistEq,VDistMa,weinerPath,...,FASA_P,FCASA+,FCASA-,VSA,dens,glob,std_dim1,std_dim2,std_dim3,vol
0,0,O=C1Cc2ccccc21,14.0,5,0.400000,0.666667,3,2.037476,6.011166,82,...,0.129128,0.326050,0.224950,138.13699,0.958985,0.031032,1.853123,1.382682,0.326444,123.18750
1,0,Clc1ccc(cc1)C1c2c(OC(N)=C1C#N)[nH][nH0]c2C(F)(F)F,20.5,9,0.444444,0.800000,5,2.954872,8.805204,1046,...,0.059026,0.847237,1.136764,296.30197,1.296254,0.258447,2.394159,1.966377,1.217135,262.82812
2,0,O=C(OC)C(=Cc1ccccc1)Cc1ccccc1,27.5,10,0.500000,1.000000,5,3.083532,8.211762,742,...,0.046707,0.817581,0.464419,296.54431,0.946322,0.162139,3.164745,1.552043,1.274330,266.62500
3,0,FC(F)(F)c1[nH0]cc2ccccc2c1,30.5,7,0.428571,0.750000,4,2.616827,7.313269,288,...,0.044688,0.417896,0.574827,187.43799,1.195242,0.018573,2.524826,1.446899,0.344087,164.95312
4,0,O=C(OC1Cc2ccccc21)C,31.0,7,0.428571,0.750000,4,2.609669,6.833154,203,...,0.103925,0.595412,0.267278,193.42802,0.978233,0.088156,2.550457,1.430062,0.757260,165.79688


In [74]:
X = np.array(df.iloc[:,3:])
y = np.array(df.iloc[:,2]).reshape(-1, 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [75]:
model = xgboost.XGBRegressor(colsample_bytree=0.4,
                 gamma=0,                 
                 learning_rate=0.07,
                 max_depth=3,
                 min_child_weight=1.5,
                 n_estimators=10000,                                                                    
                 reg_alpha=0.75,
                 reg_lambda=0.45,
                 subsample=0.6,
                 seed=42) 

In [77]:
model.fit(X_train, y_train)

[23:53:23] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=0.4, gamma=0,
       importance_type='gain', learning_rate=0.07, max_delta_step=0,
       max_depth=3, min_child_weight=1.5, missing=None, n_estimators=10000,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0.75, reg_lambda=0.45, scale_pos_weight=1, seed=42,
       silent=None, subsample=0.6, verbosity=1)

In [85]:
fi = {}
features = model.feature_importances_
for i in range(len(features)):
    fi[i] = features[i]
OrderedDict(sorted(fi.items(), key=lambda t: t[1], reverse=True))

OrderedDict([(43, 0.0960236),
             (34, 0.0628207),
             (108, 0.062278535),
             (116, 0.03989352),
             (38, 0.03740075),
             (111, 0.031613886),
             (17, 0.030831931),
             (81, 0.030521229),
             (140, 0.023070244),
             (13, 0.018352892),
             (14, 0.017345855),
             (7, 0.015753098),
             (29, 0.015477724),
             (136, 0.009563321),
             (62, 0.009230304),
             (3, 0.008989122),
             (26, 0.008674369),
             (115, 0.008242369),
             (45, 0.007898379),
             (121, 0.0067777084),
             (47, 0.006710148),
             (57, 0.0065820846),
             (31, 0.006493483),
             (8, 0.0064774477),
             (37, 0.005914722),
             (19, 0.005637877),
             (119, 0.0056032958),
             (55, 0.0053854915),
             (139, 0.0051312414),
             (61, 0.0049022553),
             (56, 0.0048718923),


In [106]:
pred = model.predict(X_test)

In [105]:
r2_score(y_test, pred)

0.5606747666696275